In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

# !pip install catboost
import catboost
# aaa

# Data 준비

나현님의 lgbm_train_df를 기준으로 실험을 진행 했다  
rating의 경우  
- 0이면 negative sample
- 1이면 positive sample

FE를 통해 더 많은 피쳐를 적용 할 수 있을 것  
"rating" column은 없으면 에러난다

In [2]:
base_df = pd.read_csv('/opt/ml/input/code/data/lgbm_train_df.csv')
base_df

,user,item,rating
0,11,4643,1.0
1,11,170,1.0
2,11,531,1.0
3,11,616,1.0
4,11,2140,1.0
...,...,...,...
10070689,138493,33672,0.0
10070690,138493,6617,0.0
10070691,138493,8446,0.0
10070692,138493,4624,0.0


FE을 통행 생성한 column들을 추가한다

In [3]:
full_year = pd.read_csv("/opt/ml/input/code/data/year_missingno.csv")
writ = pd.read_csv("/opt/ml/input/code/data/writer_inter_cnt.csv")
director = pd.read_csv("/opt/ml/input/code/data/director_inter_cnt.csv")
genre = pd.read_csv("/opt/ml/input/code/data/genre_bert_fe.csv")

merge_df = pd.merge(base_df, full_year, on='item', how='left')
merge_df = pd.merge(merge_df, writ, on='item', how='left')
merge_df = pd.merge(merge_df, director, on='item', how='left')
merge_df = pd.merge(merge_df, genre, on='item', how='left')

merge_df.isna().sum()

user               0
item               0
rating             0
year               0
writer_info        0
director_info      0
genre_embedding    0
dtype: int64

In [4]:
merge_df

,user,item,rating,year,writer_info,director_info,genre_embedding
0,11,4643,1.0,2001.0,4146.333333,2859.444444,-0.010404
1,11,170,1.0,1995.0,269.562069,583.714286,-0.010290
2,11,531,1.0,1993.0,5684.000000,430.333333,-0.010649
3,11,616,1.0,1970.0,6646.000000,1966.076923,-0.010579
4,11,2140,1.0,1982.0,2081.000000,1399.750000,-0.010351
...,...,...,...,...,...,...,...
10070689,138493,33672,0.0,2005.0,223.250000,305.166667,-0.010306
10070690,138493,6617,0.0,2003.0,269.562069,1794.250000,-0.011394
10070691,138493,8446,0.0,1949.0,10199.666667,49.000000,-0.010387
10070692,138493,4624,0.0,1989.0,21392.000000,4062.533333,-0.010443


# Dataset

먼저 numerical feature와 categorical feature를 구분하는 과정을 거친다

In [5]:
# categorical feature 목록
cat_feats = ['user', 'item', 'year']
# numerical feature 목록
nu_feats = ['writer_info', 'director_info', 'genre_embedding']

def _convert_cat_features_dtype(df):
    for cat_feat in cat_feats:
        df[cat_feat] = df[cat_feat].apply(str)
        # df[cat_feat] = df[cat_feat].astype("category")
        df[cat_feat] = df[cat_feat].astype(object)
    
def _convert_nu_features_dtype(df):
    for nu_feat in nu_feats:
        df[nu_feat] = df[nu_feat].astype(float)

# 함수에 사용할 df를 넣어주면 된다.
_convert_cat_features_dtype(merge_df)
_convert_nu_features_dtype(merge_df)

학습을 위해 train/valid split 한다

In [6]:
df_x_data = merge_df.drop(['rating'], axis=1)
df_y_data = merge_df['rating']

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_x_data, df_y_data, test_size=0.1, random_state=14)

# Model

In [7]:
model = catboost.CatBoostClassifier(
    iterations= 10, # 1번 할때가 젤 괜찮게 나옴, -> 왜지?
    random_seed= 14,
    custom_metric=["Precision","F1","Accuracy"],
    eval_metric="Precision",
    early_stopping_rounds=100,
    learning_rate=0.01,
    # loss_function="Logloss", # Binary Classification에서 적절하다고 함
    task_type="GPU",
    devices="0"
)

In [8]:
model.fit(df_x_train, df_y_train,
    cat_features=cat_feats,
    eval_set=(df_x_test, df_y_test),
    verbose=100 # 100번의 iter 마다 output을 print
)

0:	learn: 0.8258630	test: 0.8265644	best: 0.8265644 (0)	total: 43.3ms	remaining: 390ms
9:	learn: 0.8258630	test: 0.8265644	best: 0.8265644 (0)	total: 416ms	remaining: 0us
bestTest = 0.826564353
bestIteration = 0
Shrink model to first 1 iterations.


In [9]:
# model.predict_proba(df_x_test)[:,1]
a = model.predict(df_x_test)
b = df_y_test.values

cnt = 0
for i in range(221):
    if a[i] == b[i]:
        cnt+=1
print(cnt/221)

0.8054298642533937


분류 정확성이 80%정도 나온다

In [10]:
# Save model
model.save_model('/opt/ml/input/code/model/saved/catboost_model.bin') # catboost는 bin을 지원
# model.load_model('/opt/ml/input/code/model/saved/catboost_model.bin') # catboost는 bin을 지원

필요시 모델을 저장할 수 있다.  
catboost model은 .bin을 사용해야하며  
.pth등으로 저장할 수는 있으나 정식으로 지원하지는 않는다

# RecBole 모델 결과에 Catboost 적용

In [11]:
ease_top_300 = pd.read_csv('/opt/ml/input/code/Recbole/EASE_best.csv')

esae_top_300 : EASE모델을 사용하여 유저 당 300개의 영화를 추천 받은 결과이다

In [12]:
ref_df = ease_top_300
ref_df['rating'] = 0 # 에러 방지를 위한 조치, column의 의미는 없음

In [13]:
ref_df = pd.merge(ref_df,full_year, on='item', how='left')
ref_df = pd.merge(ref_df, writ, on='item', how='left')
ref_df = pd.merge(ref_df, director, on='item', how='left')
ref_df = pd.merge(ref_df, genre, on='item', how='left')
ref_df

,user,item,rating,year,writer_info,director_info,genre_embedding
0,11,4370,0,2001.0,5626.166667,4369.500000,-0.010420
1,11,4886,0,2001.0,33020.428571,16267.000000,-0.010383
2,11,47,0,1995.0,4527.200000,4853.181818,-0.010262
3,11,32587,0,2005.0,5124.250000,9632.166667,-0.010371
4,11,40815,0,2005.0,8387.333333,1669.666667,-0.010360
...,...,...,...,...,...,...,...
9407995,138493,1201,0,1966.0,7716.750000,2054.857143,-0.010670
9407996,138493,3213,0,1993.0,7450.450000,230.992337,-0.010579
9407997,138493,31685,0,2005.0,269.562069,627.125000,-0.010319
9407998,138493,5014,0,2001.0,421.714286,401.333333,-0.010508


In [14]:
cat_predict_rslt = model.predict(ref_df)
cat_predict_rslt_proba = model.predict_proba(ref_df) # [0일 확률, 1일 확률]
cat_predict_rslt_proba_1 = np.array([i[1] for i in cat_predict_rslt_proba])

In [15]:
ref_df['cat_predict_rslt'] = cat_predict_rslt
ref_df['cat_predict_rslt_proba_1'] = cat_predict_rslt_proba_1

In [16]:
ref_df

,user,item,rating,year,writer_info,director_info,genre_embedding,cat_predict_rslt,cat_predict_rslt_proba_1
0,11,4370,0,2001.0,5626.166667,4369.500000,-0.010420,1.0,0.503725
1,11,4886,0,2001.0,33020.428571,16267.000000,-0.010383,1.0,0.504724
2,11,47,0,1995.0,4527.200000,4853.181818,-0.010262,1.0,0.504724
3,11,32587,0,2005.0,5124.250000,9632.166667,-0.010371,1.0,0.504724
4,11,40815,0,2005.0,8387.333333,1669.666667,-0.010360,1.0,0.504724
...,...,...,...,...,...,...,...,...,...
9407995,138493,1201,0,1966.0,7716.750000,2054.857143,-0.010670,1.0,0.504724
9407996,138493,3213,0,1993.0,7450.450000,230.992337,-0.010579,0.0,0.497873
9407997,138493,31685,0,2005.0,269.562069,627.125000,-0.010319,1.0,0.502033
9407998,138493,5014,0,2001.0,421.714286,401.333333,-0.010508,1.0,0.502033


- cat_predict_rslt : catboost가 1(positive)로 예측했는지, 0(negative)로 예측했는지
- cat_predict_rslt_proba_1 : cat_predict_rslt가 1일 확률 -> 높을 수록 좋다

In [17]:
df_rslt_model_passed_1 = ref_df.loc[ref_df['cat_predict_rslt'] == 1].drop(['cat_predict_rslt','cat_predict_rslt_proba_1'],axis=1)

In [18]:
df_rslt_model_passed_1

,user,item,rating,year,writer_info,director_info,genre_embedding
0,11,4370,0,2001.0,5626.166667,4369.500000,-0.010420
1,11,4886,0,2001.0,33020.428571,16267.000000,-0.010383
2,11,47,0,1995.0,4527.200000,4853.181818,-0.010262
3,11,32587,0,2005.0,5124.250000,9632.166667,-0.010371
4,11,40815,0,2005.0,8387.333333,1669.666667,-0.010360
...,...,...,...,...,...,...,...
9407993,138493,7099,0,1984.0,3032.250000,2425.083333,-0.010414
9407995,138493,1201,0,1966.0,7716.750000,2054.857143,-0.010670
9407997,138493,31685,0,2005.0,269.562069,627.125000,-0.010319
9407998,138493,5014,0,2001.0,421.714286,401.333333,-0.010508


EASE 모델로 유저당 top300개 추천 받았을때 : row의 수 9408000  
catboost를 이용해서 줄임: row의 수 8514588  
총 893412개의 추천 제거

확인 결과 상위권의 cat_predict_rslt_proba값 같은 걸 볼 수 있다
- ex)  
'a'영화와 'b'영화 모두 1로 예측 되었는데 proba값도 0.51로 같은 경우 두 영화 사이의 추천 우열을 가릴 수 없다  
이런 영화가 10개를 초과하면 인간이 개입해서 제거해주어야 한다 -> 기준이 모호

따라서 catboost모델을 여러번 통과시켜서 단계적으로 제거해주는 것이 좋을 것이라 예측된다.

## 2번째 통과

In [19]:
cat_predict_rslt = model.predict(ref_df)
cat_predict_rslt_proba = model.predict_proba(ref_df) # [0일 확률, 1일 확률]
cat_predict_rslt_proba_1 = np.array([i[1] for i in cat_predict_rslt_proba])